In [1]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertConfig, BertForMaskedLM, BertForSequenceClassification
from pathlib import Path
import torch

from fastai.text import Tokenizer, Vocab
import pandas as pd
import collections
import os
from tqdm import tqdm, trange
import sys
import random
import numpy as np
import apex
from sklearn.model_selection import train_test_split

import datetime
    
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pytorch_pretrained_bert.optimization import BertAdam

from energy_bert.modeling import BertForMultiLabelSequenceClassification
from energy_bert.data import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from energy_bert.learner import BertLearner
from energy_bert.metrics import accuracy_thresh, roc_auc, fbeta, accuracy_multilabel



In [2]:
pd.set_option('display.max_colwidth', -1)
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

###  Setup paths

In [3]:
DATA_PATH = Path('../data/')
LABEL_PATH = Path('../labels/')

AUG_DATA_PATH = Path('../data/data_augmentation/')

MODEL_PATH=Path('../models/')
LOG_PATH=Path('../logs/')
MODEL_PATH.mkdir(exist_ok=True)

model_state_dict = None

BERT_PRETRAINED_PATH = Path('../models/pretrained-weights/uncased_L-12_H-768_A-12/')
# BERT_PRETRAINED_PATH = Path('../../bert_fastai/pretrained-weights/cased_L-12_H-768_A-12/')
# BERT_PRETRAINED_PATH = Path('../../bert_fastai/pretrained-weights/uncased_L-24_H-1024_A-16/')
FINETUNED_PATH = Path('../models/intent_language_model_2019-02-01_00-37-04.bin')
model_state_dict = torch.load(FINETUNED_PATH)

LOG_PATH.mkdir(exist_ok=True)

### Model parameters 

In [4]:
args = {
    "run_text": "Multilabel intent finetuned model",
    "train_size": -1,
    "val_size": -1,
    "log_path": LOG_PATH,
    "full_data_dir": DATA_PATH,
    "data_dir": DATA_PATH,
    "task_name": "multilabel_intent_classification",
    "no_cuda": False,
    "bert_model": BERT_PRETRAINED_PATH,
    "output_dir": MODEL_PATH/'output',
    "max_seq_length": 512,
    "do_train": True,
    "do_eval": True,
    "do_lower_case": True,
    "train_batch_size": 32,
    "eval_batch_size": 32,
    "learning_rate": 1e-4,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.1,
    "no_cuda": False,
    "local_rank": -1,
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "optimize_on_cpu": False,
    "fp16": True,
    "loss_scale": 128
}

In [5]:
import logging

logfile = str(LOG_PATH/'log-{}-{}.txt'.format(run_start_time, args["run_text"]))

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [6]:
logger.info(args)

02/16/2019 13:11:44 - INFO - root -   {'run_text': 'Multilabel intent finetuned model', 'train_size': -1, 'val_size': -1, 'log_path': PosixPath('../logs'), 'full_data_dir': PosixPath('../data'), 'data_dir': PosixPath('../data'), 'task_name': 'multilabel_intent_classification', 'no_cuda': False, 'bert_model': PosixPath('../models/pretrained-weights/uncased_L-12_H-768_A-12'), 'output_dir': PosixPath('../models/output'), 'max_seq_length': 512, 'do_train': True, 'do_eval': True, 'do_lower_case': True, 'train_batch_size': 32, 'eval_batch_size': 32, 'learning_rate': 0.0001, 'num_train_epochs': 12.0, 'warmup_proportion': 0.1, 'local_rank': -1, 'seed': 42, 'gradient_accumulation_steps': 1, 'optimize_on_cpu': False, 'fp16': True, 'loss_scale': 128}


In [7]:
tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=args['do_lower_case'])

02/16/2019 13:11:44 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file ../models/pretrained-weights/uncased_L-12_H-768_A-12/vocab.txt


In [8]:
device = torch.device('cuda')

In [9]:
# train_features = convert_examples_to_features(train_examples, label_list=labels, tokenizer=tokenizer, max_seq_length=args['max_seq_length'])

In [10]:
databunch = BertDataBunch(args['data_dir'], LABEL_PATH, tokenizer, train_file='train_2.csv', val_file='val_2.csv',
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], multi_gpu=True, multi_label=True)

In [11]:
num_labels = len(databunch.labels)

In [12]:
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})
metrics.append({'name': 'accuracy_single', 'function': accuracy_multilabel})

In [13]:
learner = BertLearner.from_pretrained_model(databunch, BERT_PRETRAINED_PATH, metrics, device, logger, 
                                            finetuned_wgts_path=FINETUNED_PATH, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], multi_label=True)

02/16/2019 13:11:52 - INFO - pytorch_pretrained_bert.modeling -   loading archive file ../models/pretrained-weights/uncased_L-12_H-768_A-12 from cache at ../models/pretrained-weights/uncased_L-12_H-768_A-12
02/16/2019 13:11:52 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

02/16/2019 13:11:57 - INFO - pytorch_pretrained_bert.modeling -   Weights of BertForMultiLabelSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
02/16/2019 13:11:57 - INFO - pytorch_pretrained_bert.modeling -   Weights from pretrained model not used in BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.de

In [14]:
learner.fit(20, lr=args['learning_rate'])

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


02/16/2019 13:14:01 - INFO - root -   Loss after epoch 0 - 0.13766268216646635
02/16/2019 13:14:01 - INFO - root -   Running evaluation


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


02/16/2019 13:14:22 - INFO - root -   Eval results:
02/16/2019 13:14:22 - INFO - root -     eval_loss = 0.025739229642427884
02/16/2019 13:14:22 - INFO - root -     metrics = {'accuracy_thresh': 0.9960000514984131, 'roc_auc': 0.7781234972067597, 'fbeta': 0.0, 'accuracy_single': 0.04839874789308933}
02/16/2019 13:14:22 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:16:10 - INFO - root -   Loss after epoch 1 - 0.024047323373647837
02/16/2019 13:16:10 - INFO - root -   Running evaluation


02/16/2019 13:16:31 - INFO - root -   Eval results:
02/16/2019 13:16:31 - INFO - root -     eval_loss = 0.023202045147235575
02/16/2019 13:16:31 - INFO - root -     metrics = {'accuracy_thresh': 0.9960000514984131, 'roc_auc': 0.8197863106958669, 'fbeta': 0.0, 'accuracy_single': 0.040693474596677104}
02/16/2019 13:16:31 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:18:19 - INFO - root -   Loss after epoch 2 - 0.02314312274639423
02/16/2019 13:18:19 - INFO - root -   Running evaluation


...

02/16/2019 13:46:38 - INFO - root -   Eval results:
02/16/2019 13:46:38 - INFO - root -     eval_loss = 0.007184366079477164
02/16/2019 13:46:38 - INFO - root -     metrics = {'accuracy_thresh': 0.9982750415802002, 'roc_auc': 0.9814975299377026, 'fbeta': 0.7523878216743469, 'accuracy_single': 0.758728629906092}
02/16/2019 13:46:38 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:48:26 - INFO - root -   Loss after epoch 16 - 0.0011648611380503729
02/16/2019 13:48:26 - INFO - root -   Running evaluation


02/16/2019 13:48:47 - INFO - root -   Eval results:
02/16/2019 13:48:47 - INFO - root -     eval_loss = 0.007150620680588943
02/16/2019 13:48:47 - INFO - root -     metrics = {'accuracy_thresh': 0.9983164668083191, 'roc_auc': 0.981643077321721, 'fbeta': 0.7561601996421814, 'accuracy_single': 0.7635444257163496}
02/16/2019 13:48:47 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:50:35 - INFO - root -   Loss after epoch 17 - 0.0010135964705393864
02/16/2019 13:50:35 - INFO - root -   Running evaluation


02/16/2019 13:50:55 - INFO - root -   Eval results:
02/16/2019 13:50:55 - INFO - root -     eval_loss = 0.007274128840519832
02/16/2019 13:50:55 - INFO - root -     metrics = {'accuracy_thresh': 0.9982634782791138, 'roc_auc': 0.9811817399988816, 'fbeta': 0.7500200867652893, 'accuracy_single': 0.7620996869732723}
02/16/2019 13:50:55 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:52:43 - INFO - root -   Loss after epoch 18 - 0.0009088511650378888
02/16/2019 13:52:43 - INFO - root -   Running evaluation


02/16/2019 13:53:03 - INFO - root -   Eval results:
02/16/2019 13:53:03 - INFO - root -     eval_loss = 0.007326052739070012
02/16/2019 13:53:03 - INFO - root -     metrics = {'accuracy_thresh': 0.9982759952545166, 'roc_auc': 0.9805259597573299, 'fbeta': 0.7559595108032227, 'accuracy_single': 0.7616181073922466}
02/16/2019 13:53:03 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:54:52 - INFO - root -   Loss after epoch 19 - 0.0008496020848934467
02/16/2019 13:54:52 - INFO - root -   Running evaluation


02/16/2019 13:55:13 - INFO - root -   Eval results:
02/16/2019 13:55:13 - INFO - root -     eval_loss = 0.007346945542555589
02/16/2019 13:55:13 - INFO - root -     metrics = {'accuracy_thresh': 0.9982894659042358, 'roc_auc': 0.9803986506054424, 'fbeta': 0.7549563050270081, 'accuracy_single': 0.7625812665542981}
02/16/2019 13:55:13 - INFO - root -   --------------------------------------------------------------------------------


In [15]:
learner.save_and_reload(MODEL_PATH, 'intent_multilabel_classification_{}'.format(run_start_time))

02/16/2019 13:55:14 - INFO - pytorch_pretrained_bert.modeling -   loading archive file ../models/pretrained-weights/uncased_L-12_H-768_A-12 from cache at ../models/pretrained-weights/uncased_L-12_H-768_A-12
02/16/2019 13:55:14 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [16]:
texts = ["please contact me on a different email address"]
learner.predict_batch(texts)

[[('change-email', 0.9453),
  ('enquire-vague', 0.00656),
  ('enquire-account_online', 0.004772),
  ('report-no_energy', 0.003172),
  ('report-contact', 0.00291),
  ('enquire-call_back', 0.002388),
  ('report-home_move', 0.002165),
  ('request-advisor', 0.00192),
  ('problem-heating', 0.001854),
  ('change-address', 0.00181),
  ('change-account_detail', 0.001789),
  ('cancel-vague', 0.001727),
  ('vague-fraud', 0.001636),
  ('problem-drain', 0.001549),
  ('change-direct_debit', 0.001422),
  ('request-appointment_emergency', 0.001393),
  ('report-appointment_missed', 0.00132),
  ('enquire-appointment_today', 0.001315),
  ('enquire-password', 0.0011425),
  ('problem-account_setup', 0.001125),
  ('enquire-annual_service', 0.001095),
  ('problem-vague', 0.001081),
  ('enquire-power_cut', 0.001069),
  ('change-ownership', 0.001009),
  ('enquire-authority', 0.000959),
  ('change-appointment_boiler', 0.0009074),
  ('problem-boiler', 0.0009003),
  ('problem-home_move', 0.0008726),
  ('request-

In [17]:
# "../models/intent_multilabel_classification_2019-02-11_13-41-32.bin"

In [18]:
# learner.fit(5, lr=args['learning_rate'])

In [19]:
learner.fit(5, lr=args['learning_rate']/10)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


02/16/2019 13:57:10 - INFO - root -   Loss after epoch 0 - 0.0008339344308926509
02/16/2019 13:57:10 - INFO - root -   Running evaluation


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


02/16/2019 13:57:30 - INFO - root -   Eval results:
02/16/2019 13:57:30 - INFO - root -     eval_loss = 0.007542654184194712
02/16/2019 13:57:30 - INFO - root -     metrics = {'accuracy_thresh': 0.998266339302063, 'roc_auc': 0.9795117593451249, 'fbeta': 0.7587286829948425, 'accuracy_single': 0.7616181073922466}
02/16/2019 13:57:30 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 13:59:18 - INFO - root -   Loss after epoch 1 - 0.000742410237972553
02/16/2019 13:59:18 - INFO - root -   Running evaluation


02/16/2019 13:59:39 - INFO - root -   Eval results:
02/16/2019 13:59:39 - INFO - root -     eval_loss = 0.007621398338904748
02/16/2019 13:59:39 - INFO - root -     metrics = {'accuracy_thresh': 0.9982827305793762, 'roc_auc': 0.979486848438543, 'fbeta': 0.7550765872001648, 'accuracy_single': 0.7604141584396822}
02/16/2019 13:59:39 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 14:01:26 - INFO - root -   Loss after epoch 2 - 0.0006539528186504657
02/16/2019 14:01:26 - INFO - root -   Running evaluation


02/16/2019 14:01:47 - INFO - root -   Eval results:
02/16/2019 14:01:47 - INFO - root -     eval_loss = 0.007682389479417067
02/16/2019 14:01:47 - INFO - root -     metrics = {'accuracy_thresh': 0.9982962012290955, 'roc_auc': 0.9788876366145745, 'fbeta': 0.7586483955383301, 'accuracy_single': 0.7640260052973754}
02/16/2019 14:01:47 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 14:03:35 - INFO - root -   Loss after epoch 3 - 0.0006072790576861455
02/16/2019 14:03:35 - INFO - root -   Running evaluation


02/16/2019 14:03:55 - INFO - root -   Eval results:
02/16/2019 14:03:55 - INFO - root -     eval_loss = 0.007748603820800781
02/16/2019 14:03:55 - INFO - root -     metrics = {'accuracy_thresh': 0.9982942938804626, 'roc_auc': 0.9783753737681776, 'fbeta': 0.7582069635391235, 'accuracy_single': 0.764748374668914}
02/16/2019 14:03:55 - INFO - root -   --------------------------------------------------------------------------------
02/16/2019 14:05:43 - INFO - root -   Loss after epoch 4 - 0.0005852057383610652
02/16/2019 14:05:43 - INFO - root -   Running evaluation


02/16/2019 14:06:03 - INFO - root -   Eval results:
02/16/2019 14:06:03 - INFO - root -     eval_loss = 0.0077564092782827524
02/16/2019 14:06:03 - INFO - root -     metrics = {'accuracy_thresh': 0.9982962012290955, 'roc_auc': 0.9784281100520467, 'fbeta': 0.7575648427009583, 'accuracy_single': 0.7654707440404527}
02/16/2019 14:06:03 - INFO - root -   --------------------------------------------------------------------------------
